In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Dataset, random_split, TensorDataset
import torch.nn as nn
from sklearn.metrics import mean_squared_error, mean_absolute_error, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight
import itertools


In [3]:
X_train = pd.read_csv("Training_Validation_Test_Datasets/task2_input_train.csv", index_col=0)
X_validate = pd.read_csv("Training_Validation_Test_Datasets/task2_input_validate.csv", index_col=0)
X_test = pd.read_csv("Training_Validation_Test_Datasets/task2_input_test.csv", index_col=0)

y_train = pd.read_csv("Training_Validation_Test_Datasets/task2_output_train.csv")["phq_sum"]
y_validate = pd.read_csv("Training_Validation_Test_Datasets/task2_output_validate.csv")["phq_sum"]
y_test = pd.read_csv("Training_Validation_Test_Datasets/task2_output_test.csv")["phq_sum"]

In [ ]:
# Initialize the scaler
scaler = StandardScaler()

# Fit the scaler on data and transform
X_train_scaled = scaler.fit_transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Convert inputs to tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
X_validate_tensor = torch.tensor(X_validate_scaled, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)

# Convert outputs (labels) to tensors
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
y_validate_tensor = torch.tensor(y_validate.values, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

In [ ]:
# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
validate_dataset = TensorDataset(X_validate_tensor, y_validate_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [ ]:
# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validate_loader = DataLoader(validate_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
class PHQModel(nn.Module):
    def __init__(self, input_size, hidden_num, hidden_size):
        super(PHQModel, self).__init__()
        
        self.hidden_layers = nn.ModuleList()
        if hidden_num > 0:
            self.hidden_layers.append(nn.Linear(input_size, hidden_size)) # first layer
            
            for _ in range(hidden_num - 1):
                self.hidden_layers.append(nn.Linear(hidden_size, hidden_size))
                
            reduced_size = hidden_size // 2
            self.hidden_layers.append(nn.Linear(hidden_size, reduced_size))
            self.output_layer = nn.Linear(reduced_size, 1)
        else:
            # If no hidden layers, output layer connects directly to input
            self.output_layer = nn.Linear(input_size, 1)
        
        # Define the activation function
        self.relu = nn.ReLU()
        # Add dropout
        self.dropout = nn.Dropout(0.2)
            
    def forward(self, x):
        if len(self.hidden_layers) == 0:
            # No hidden layers, pass directly to output layer
            x = self.output_layer(x)
        else:
            # Pass through each hidden layer with ReLU activation
            for layer in self.hidden_layers:
                x = self.relu(layer(x))
                # Apply dropout
                x = self.dropout(x)
            # Then pass through the output layer
            x = self.output_layer(x)
        return x

In [ ]:
def network_execution(hidden_num, hidden_size, X_train, train_loader, validate_loader, test_loader):
    # Initialize the model
    input_size = X_train.shape[1]  # Number of features
    model = PHQModel(input_size, hidden_num, hidden_size)

    # Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # Training with Learning Rate Scheduler
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.1)
    num_epochs = 100

    for epoch in range(num_epochs):
        # Training phase
        model.train()  # Set the model to training mode
        running_loss = 0.0

        for inputs, labels in train_loader:
            optimizer.zero_grad()  
            outputs = model(inputs)  
            loss = criterion(outputs, labels)  
            loss.backward()  
            optimizer.step() 
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)  # Average training loss

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in validate_loader:  # Use validate_loader instead of test_loader
                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()

        val_loss = val_loss / len(validate_loader)  # Average validation loss

        lr = scheduler.get_last_lr()[0]
        # Step the learning rate scheduler
        scheduler.step(val_loss)

        # Print progress every epoch or at specific intervals
        # if epoch == 99:
        #     print(f"Epoch {epoch+1}/{num_epochs}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

    # Evaluation
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            all_preds.extend(outputs.numpy().flatten())
            all_labels.extend(labels.numpy().flatten())

    mse = mean_squared_error(all_labels, all_preds)
    mae = mean_absolute_error(all_labels, all_preds)

    # print(f"Test MSE: {mse}, Test MAE: {mae}")
    return mse, all_preds

In [5]:
# Hyperparameter Tuning

hidden_numbers = [1, 2, 3, 4, 5, 6]
hidden_sizes = [16, 32, 64, 128, 256, 512]
combinations = list(itertools.product(hidden_numbers, hidden_sizes))

mse_results = []

for hidden_num, hidden_size in combinations:
    print(f"Training model: Hidden Layers = {hidden_num}, Hidden Size = {hidden_size}")
    
    mse, _ = network_execution(hidden_num, hidden_size, X_train, train_loader, validate_loader) 
    mse_results.append((mse, (hidden_num, hidden_size)))

best_result = min(mse_results, key=lambda x: x[0])
worst_result = max(mse_results, key=lambda x: x[0])

print("\n✅ Hyperparameter Tuning Results:")
print(f"Best Parameters: Hidden Layers = {best_result[1][0]}, Hidden Size = {best_result[1][1]}")
print(f"Best Validation MSE: {best_result[0]:.4f}")

print(f"Worst Parameters: Hidden Layers = {worst_result[1][0]}, Hidden Size = {worst_result[1][1]}")
print(f"Worst Validation MSE: {worst_result[0]:.4f}")


Training model: Hidden Layers = 1, Hidden Size = 16


NameError: name 'network_execution' is not defined